#Created three models

1. Model0 --> Baseline model
2. Model1 --> Introduced Non Linearity
3. Model2 --> CNN architecture

#FashionMNIST dataset

In [1]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

print(torch.__version__)
print(torchvision.__version__)

2.3.0+cu121
0.18.0+cu121


In [2]:
#using the built-in dataset

#setup the training data
from torchvision import datasets

train_data = datasets.FashionMNIST(
    root = "data", #where to download the data to--
    train = True, #do we want the training dataset, if False we get the testing dataset
    download = True, #to get the dataset
    transform = torchvision.transforms.ToTensor(), #convert the data to tensors
    target_transform = None #how we want to transform the labels/targets
)


test_data = datasets.FashionMNIST(
    root="data",
    train = False,
    download = True,
    transform = ToTensor(),
    target_transform = None
)

100%|██████████| 26421880/26421880 [00:08<00:00, 2981999.71it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 270807.74it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5039643.02it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5076952.03it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
#checking out the parameters or the attributes of the data
len(train_data) , len(test_data)

(60000, 10000)

In [4]:
#see the first training example
image , label = train_data[0]
image , label

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0510,
           0.2863, 0.0000, 0.0000, 0.0039, 

In [5]:
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [6]:
class_to_index = train_data.class_to_idx #map class labels (or categories) to their corresponding indices
class_to_index

{'T-shirt/top': 0,
 'Trouser': 1,
 'Pullover': 2,
 'Dress': 3,
 'Coat': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'Bag': 8,
 'Ankle boot': 9}

In [7]:
train_data.targets

tensor([9, 0, 0,  ..., 3, 0, 5])

In [8]:
#checking the shape of the image
image.shape , label #label is associated with the category

(torch.Size([1, 28, 28]), 9)

In [9]:
print(f'image shape is: {image.shape} --> [color channel, height, width]')
print(f'image label is: {class_names[label]}')

image shape is: torch.Size([1, 28, 28]) --> [color channel, height, width]
image label is: Ankle boot


#Prepare the DataLoader

In [10]:
from torch.utils.data import DataLoader

batch_size = 32 #setup the batch size hyperparameter

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=batch_size,
                              shuffle=True) #for randomness

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=batch_size,
                             shuffle=False)

train_dataloader , test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7dcbb07f77c0>,
 <torch.utils.data.dataloader.DataLoader at 0x7dcbb07f78e0>)

In [11]:
#to check what we have created
print(f'Dataloaders: {train_dataloader , test_dataloader}')
print(f'Length of the training dataloader is: {len(train_dataloader)} with batches of {batch_size}')
print(f'Length of the testing dataloader is: {len(test_dataloader)} with batches of {batch_size}')


Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7dcbb07f77c0>, <torch.utils.data.dataloader.DataLoader object at 0x7dcbb07f78e0>)
Length of the training dataloader is: 1875 with batches of 32
Length of the testing dataloader is: 313 with batches of 32


In [12]:
train_features_batch , train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape , train_labels_batch.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

#building a baseline model - Model 0



In [13]:
#create a flatten layer -->a flatten layer is a type of layer that is used to reshape the input data into a one-dimensional array
flatten_model = nn.Flatten()

#get a single sample
x = train_features_batch[0]
x.shape

#now lets flatten the sample
output = flatten_model(x)

print(f'shape before flattening: {x.shape}, it consisted of [color_channels, height, width]')
print(f'shape after flattening: {output.shape}, it now consists of [color_channels, height*wodth]')

shape before flattening: torch.Size([1, 28, 28]), it consisted of [color_channels, height, width]
shape after flattening: torch.Size([1, 784]), it now consists of [color_channels, height*wodth]


In [14]:
from torch import nn
class FashionMNISTModel(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = input_shape,
                  out_features = hidden_units),
        nn.Linear(in_features = hidden_units,
                  out_features = output_shape)
    )

  def forward(self,x):
    return self.layer_stack(x)




In [15]:
torch.manual_seed(42)

model0 = FashionMNISTModel(
    input_shape = 784,  #28*28
    hidden_units = 10,
    output_shape=len(class_names) #one for every class
).to('cpu')

model0

FashionMNISTModel(
  (layer_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [16]:
trial = torch.rand([1,1,28,28])
model0(trial)

tensor([[-0.0315,  0.3171,  0.0531, -0.2525,  0.5959,  0.2112,  0.3233,  0.2694,
         -0.1004,  0.0157]], grad_fn=<AddmmBackward0>)

In [17]:
import requests
from pathlib import Path

if Path('helper_functions.py').is_file():
  print('helper function already exists, skipping download...')
else:
  print('Downloading the helper_functions.py')
  request = requests.get('https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py')
  with open('helper_functions.py','wb') as f:
    f.write(request.content)

In [18]:
#import the accuracy metric from the file
from helper_functions import accuracy_fn

#set up the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model0.parameters(),
                            lr=0.1)

In [19]:
from timeit import default_timer as timer

def print_train_time(start: float,
                     end:float,
                     device: torch.device = None):
  #prints the diff b/w start and stop time
  total_time = end - start
  print(f'train time on {device}: {total_time:.3f} seconds')
  return total_time

In [20]:
start_time = timer()

end_time = timer()
print_train_time(start=start_time , end=end_time, device='cpu')

train time on cpu: 0.000 seconds


5.5809000002682296e-05

In [21]:
#import tqdm for progress bar
from tqdm.auto import tqdm

#set the seed and start the timer
torch.manual_seed(42)
train_time_start_on_cpu = timer()

#set the number of epochs
epochs = 3

#create a training and testing loop
for epoch in tqdm(range(epochs)):
  print(f'epoch: {epoch}\n------')

  ####training
  train_loss = 0 #cumulating values
  #add a loop to loop through training batches
  for batch, (x,y) in enumerate(train_dataloader): #x is the image and y is the label
    model0.train()
    #1. forward pass
    y_pred = model0(x)

    #2. calculate the loss (per batch)
    loss = loss_fn(y_pred , y)
    train_loss+= loss #to add loss of every batch

    #3. optimizer zero grad
    optimizer.zero_grad()

    #4. loss backward
    loss.backward()

    #5. optimizer step
    optimizer.step()

    if batch%400 == 0:
       print(f'looked at {batch * len(x)}/{len(train_dataloader.dataset)} samples.')

  #divide total train loss by length of the train dataloader
  train_loss /= len(train_dataloader)


  ###testing
  test_loss , test_acc = 0,0
  model0.eval()
  with torch.inference_mode():
    for x_test,y_test in test_dataloader:

      #1. forward pass
      test_pred = model0(x_test)

      #2. calculate the loss
      test_loss+= loss_fn(test_pred , y_test)

      #3. calculate the accuracy
      test_acc+= accuracy_fn(y_true= y_test, y_pred = test_pred.argmax(dim=1))

    #calculate the test loss average per batch
    test_loss /= len(test_dataloader)

    #calculate the test acc average per batch
    test_acc/=len(test_dataloader)

  #print out everything
  print(f'\ntrain loss: {train_loss:.4f} | test loss:{test_loss:.4f} | test acc : {test_acc:.4f}%')

#calculate the training time
train_time_end_on_cpu = timer()
total_train_time_model0 = print_train_time(start = train_time_start_on_cpu,
                                           end = train_time_end_on_cpu,
                                           device = str(next(model0.parameters()).device))

  0%|          | 0/3 [00:00<?, ?it/s]

epoch: 0
------
looked at 0/60000 samples.
looked at 12800/60000 samples.
looked at 25600/60000 samples.
looked at 38400/60000 samples.
looked at 51200/60000 samples.

train loss: 0.5904 | test loss:0.5095 | test acc : 82.0387%
epoch: 1
------
looked at 0/60000 samples.
looked at 12800/60000 samples.
looked at 25600/60000 samples.
looked at 38400/60000 samples.
looked at 51200/60000 samples.

train loss: 0.4763 | test loss:0.4799 | test acc : 83.1969%
epoch: 2
------
looked at 0/60000 samples.
looked at 12800/60000 samples.
looked at 25600/60000 samples.
looked at 38400/60000 samples.
looked at 51200/60000 samples.

train loss: 0.4550 | test loss:0.4766 | test acc : 83.4265%
train time on cpu: 33.586 seconds


#making predictions and getting model results


In [22]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn):
  #returns a dict containing the results of the model predictions on data_loader
  loss ,  acc = 0 , 0
  model.eval()
  with torch.inference_mode():
    for x,y in tqdm(data_loader):
      y_pred = model(x)  #predictions

      loss += loss_fn(y_pred , y)
      acc+= accuracy_fn(y_true=y,
                        y_pred=y_pred.argmax(dim=1))

    #scale loss and acc to find the average loss and acc per batch
    loss/= len(data_loader)
    acc/= len(data_loader)

  return {'model_name:' : model.__class__.__name__,
          'model_loss:' : loss.item(),
          'model_acc:' : acc}

#calculate results
model_0_results = eval_model(model=model0,
                            data_loader=test_dataloader,
                            loss_fn=loss_fn,
                            accuracy_fn=accuracy_fn)
model_0_results



  0%|          | 0/313 [00:00<?, ?it/s]

{'model_name:': 'FashionMNISTModel',
 'model_loss:': 0.47663894295692444,
 'model_acc:': 83.42651757188499}

# Introducing Non - Linearity : Model 1

In [23]:
# Setup device agnostic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [24]:
class FashionMNISTModelV1(nn.Module):
  def __init__(self,
               input_shape : int,
               hidden_units : int,
               output_shape: int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = input_shape,
                  out_features = hidden_units),
        nn.ReLU(),
        nn.Linear(in_features = hidden_units,
                  out_features = output_shape),
        nn.ReLU(),

    )

  def forward(self, x: torch.Tensor):
    return self.layer_stack(x)


#this model is only different because of two Relu functions added

In [25]:
torch.manual_seed(42)
model_1 = FashionMNISTModelV1(input_shape = 784,
                             hidden_units = 10,
                             output_shape = len(class_names)).to(device)

next(model_1.parameters()).device

device(type='cpu')

In [26]:
#creating a loss function
from helper_functions import accuracy_fn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_1.parameters() ,
                            lr = 0.1)

#Creating a function for the training and testing loop

In [27]:
#training_loop = train_step
#testing_loop = test_step


In [28]:
def train_step(model: torch.nn.Module,
                data_loader: torch.utils.data.DataLoader,
                loss_fn: torch.nn.Module,
                optimizer: torch.optim.Optimizer,
                accuracy_fn,
                device: torch.device = device):
  train_loss , train_acc = 0 , 0
  model.to(device)
  #model.train()


  #add a loop to loop through training batches
  for batch, (X,y) in enumerate(train_dataloader): #x is the image and y is the label
      #target device
      X , y = X.to(device) , y.to(device)

      #1. forward pass
      y_pred = model(X)

      #2. calculate the loss (per batch)
      loss = loss_fn(y_pred , y)
      train_loss+= loss #to add loss of every batch
      train_acc += accuracy_fn(y_true = y,
                                y_pred = y_pred.argmax(dim=1))
      #3. optimizer zero grad
      optimizer.zero_grad()

      #4. loss backward
      loss.backward()

      #5. optimizer step
      optimizer.step()


  #divide total train loss by length of the train dataloader
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f'TRAIN LOSS: {train_loss:.5f} | TRAIN ACC: {train_acc:.2f}')

In [29]:
#testing loop
def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
  test_loss , test_acc =0,0

  #put the model to eval mode
  model.eval()

  #turn on the inference mode
  with torch.inference_mode():
    for X, y in data_loader:
      #send the data to the target device
      X, y = X.to(device) , y.to(device)

      #forward pass
      test_pred = model(X)

      #claculate the loss/acc
      test_loss += loss_fn(test_pred , y)
      test_acc += accuracy_fn(y_true = y,
                              y_pred = test_pred.argmax(dim=1))

      #adjust the matrics and print out the
      test_loss /= len(data_loader)
      test_acc /= len(data_loader)
      print(f'Test loss: {test_loss:.5f} | Test Acc: {test_acc: .2f}')

In [30]:
torch.manual_seed(42)

#measure time
from timeit import default_timer as timer
train_time_start_on_gpu = timer()

#set epochs
epochs = 3

#create a optimization and evaluation loop using train step and test step
for epoch in tqdm(range(epochs)):
  print(f'Epoch: {epoch}\n--------')
  train_step(model = model_1,
             data_loader = train_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             accuracy_fn = accuracy_fn,
             device = device)


  test_step(model = model_1,
            data_loader = test_dataloader,
            loss_fn = loss_fn,
            accuracy_fn = accuracy_fn,
            device = device)

train_time_end_on_gpu = timer()
total_train_time_model_1 = print_train_time(start=train_time_start_on_gpu,
                                            end = train_time_end_on_gpu,
                                            device = device)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
--------
TRAIN LOSS: 1.09199 | TRAIN ACC: 61.34
Test loss: 0.00306 | Test Acc:  0.22
Test loss: 0.00278 | Test Acc:  0.21
Test loss: 0.00261 | Test Acc:  0.21
Test loss: 0.00220 | Test Acc:  0.24
Test loss: 0.00204 | Test Acc:  0.25
Test loss: 0.00254 | Test Acc:  0.21
Test loss: 0.00241 | Test Acc:  0.23
Test loss: 0.00335 | Test Acc:  0.20
Test loss: 0.00262 | Test Acc:  0.21
Test loss: 0.00309 | Test Acc:  0.23
Test loss: 0.00293 | Test Acc:  0.18
Test loss: 0.00291 | Test Acc:  0.22
Test loss: 0.00222 | Test Acc:  0.24
Test loss: 0.00216 | Test Acc:  0.24
Test loss: 0.00408 | Test Acc:  0.18
Test loss: 0.00288 | Test Acc:  0.22
Test loss: 0.00292 | Test Acc:  0.20
Test loss: 0.00313 | Test Acc:  0.20
Test loss: 0.00333 | Test Acc:  0.20
Test loss: 0.00337 | Test Acc:  0.19
Test loss: 0.00206 | Test Acc:  0.25
Test loss: 0.00338 | Test Acc:  0.21
Test loss: 0.00325 | Test Acc:  0.24
Test loss: 0.00278 | Test Acc:  0.22
Test loss: 0.00220 | Test Acc:  0.24
Test loss: 0.00278

In [31]:
model_0_results #baseline / previous model


{'model_name:': 'FashionMNISTModel',
 'model_loss:': 0.47663894295692444,
 'model_acc:': 83.42651757188499}

In [32]:
total_train_time_model0

33.58553652699999

In [33]:
#getting model1 results
model_1_results = eval_model(model = model_1,
                             data_loader = test_dataloader,
                             loss_fn = loss_fn,
                             accuracy_fn = accuracy_fn)

model_1_results

  0%|          | 0/313 [00:00<?, ?it/s]

{'model_name:': 'FashionMNISTModelV1',
 'model_loss:': 0.6850009560585022,
 'model_acc:': 75.01996805111821}

#CNN : model 2

Conv block1 --> conv2d , relu , conv2d , relu , maxpool2d

Conv block2 --> conv2d , relu , conv2d , relu , maxpool2d

classifier --> flatten, linear


In [34]:
#CNN

class FashionMNISTModelV2(nn.Module):
  def __init__(self, input_shape: int,
               hidden_units : int,
               output_shape: int):
    super().__init__()
    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels = input_shape,
                out_channels = hidden_units,
                kernel_size=3,
                stride=1,   #the no. of pixels by which the filter shifts
                padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_units,
                  out_channels = hidden_units,
                  kernel_size=3,
                  stride=1,   #the no. of pixels by which the filter shifts
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2 , stride=2)
    )
    self.block_2 = nn.Sequential(
        nn.Conv2d(in_channels = hidden_units,
                  out_channels = hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_units,
                  out_channels = hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units*7*7,
                  out_features = output_shape)
    )
  def forward(self, x: torch.Tensor):
    x= self.block_1(x)
    #print(x.shape)
    x = self.block_2(x)
    #print(x.shape)
    x = self.classifier(x)
    return x




In [35]:
torch.manual_seed(42)
model_2 = FashionMNISTModelV2(input_shape = 1,
                              hidden_units = 10,
                              output_shape = len(class_names)).to(device)
model_2

FashionMNISTModelV2(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=490, out_features=10, bias=True)
  )
)

In [36]:
#the model is trying to take the input and learn a compressed representation from each
#of these layers.

#we will flatten the layers before putting them to output layers.

# Learning about conv2d

conv2d stands for "2-dimensional convolution" and is a fundamental operation in convolutional neural networks (CNNs), particularly useful for image processing tasks. It is used to apply a convolution operation to 2D data, such as images, which have two spatial dimensions (height and width) and a depth dimension (number of channels, like RGB for color images).

In [37]:
#creating a dummy data
images = torch.rand(size=(32,3,64,64))
test_image = images[0]

print(f'image batch shape: {images.shape}')
print(f'single image shape: {test_image.shape}')
print(f'Test image: \n{test_image}')

image batch shape: torch.Size([32, 3, 64, 64])
single image shape: torch.Size([3, 64, 64])
Test image: 
tensor([[[0.1864, 0.7294, 0.5354,  ..., 0.5492, 0.0344, 0.6408],
         [0.2555, 0.1222, 0.2993,  ..., 0.1025, 0.0051, 0.4813],
         [0.6474, 0.2053, 0.8934,  ..., 0.0936, 0.5672, 0.3089],
         ...,
         [0.3974, 0.4522, 0.3594,  ..., 0.6474, 0.1301, 0.0709],
         [0.1341, 0.9047, 0.1645,  ..., 0.0250, 0.3867, 0.4522],
         [0.8581, 0.4958, 0.6895,  ..., 0.9223, 0.6003, 0.8415]],

        [[0.0634, 0.5660, 0.9669,  ..., 0.6836, 0.3126, 0.4321],
         [0.5317, 0.9469, 0.8746,  ..., 0.3124, 0.3441, 0.6122],
         [0.8539, 0.3364, 0.4146,  ..., 0.2116, 0.4786, 0.8728],
         ...,
         [0.8372, 0.7806, 0.1906,  ..., 0.0859, 0.6138, 0.1119],
         [0.8947, 0.3029, 0.6021,  ..., 0.2203, 0.3237, 0.5624],
         [0.2022, 0.9473, 0.2036,  ..., 0.9000, 0.9285, 0.6965]],

        [[0.3733, 0.2127, 0.3968,  ..., 0.4449, 0.5642, 0.8521],
         [0.5600, 0

In [38]:
torch.manual_seed(42)
#create a single conv 2d layer
conv_layer = nn.Conv2d(in_channels = 3,
                       out_channels = 10,
                       kernel_size=(3,3),
                       stride=1,
                       padding=0
                       )

In [39]:
#pass the data through the convolutional layer
conv_output = conv_layer(test_image)
conv_output

tensor([[[ 0.5678,  0.6005,  0.3279,  ...,  0.3851,  0.2667,  0.1858],
         [ 0.3750,  0.4722,  0.5138,  ...,  0.1165,  0.2448,  0.2283],
         [ 0.3052,  0.2300,  0.5079,  ...,  0.7252,  0.2942,  0.2507],
         ...,
         [ 0.3870,  0.4742,  0.6816,  ...,  0.3085,  0.4836,  0.1088],
         [ 0.6347,  0.7210,  0.5226,  ...,  0.2876,  0.2743,  0.3371],
         [ 0.3647,  0.3771,  0.4698,  ...,  0.4586,  0.5726,  0.4925]],

        [[-0.0412, -0.0171,  0.0887,  ..., -0.1623,  0.1061, -0.0369],
         [ 0.1408,  0.0481,  0.0737,  ..., -0.5203, -0.0532, -0.4455],
         [-0.2214,  0.0543, -0.2016,  ..., -0.0053, -0.3152,  0.2101],
         ...,
         [-0.1686,  0.2848,  0.0603,  ..., -0.2487, -0.0468, -0.2221],
         [-0.1290, -0.0189, -0.2844,  ..., -0.0653, -0.2479, -0.1823],
         [-0.1067, -0.0900,  0.1174,  ...,  0.0386,  0.1390,  0.1859]],

        [[ 0.0022, -0.0055,  0.0555,  ...,  0.0398,  0.1160, -0.1060],
         [-0.0174,  0.2587,  0.0324,  ...,  0

In [40]:
test_image.shape

torch.Size([3, 64, 64])

In [41]:
rand_image_tensor = torch.randn(size=(1,28,28))
rand_image_tensor.shape

torch.Size([1, 28, 28])

#Learning about MaxPool2d


max_pool2d stands for "2-dimensional max pooling" and is another key operation in convolutional neural networks (CNNs), often used after convolutional layers. Its primary purpose is to reduce the spatial dimensions (height and width) of the input feature maps, while retaining the most important information.

In [42]:
print(f'test image original shape: {test_image.shape}')

#create a sample maxpool layer
max_pool_layer = nn.MaxPool2d(kernel_size=2)

#pass the data through just the conv layer first
test_image_through_conv = conv_layer(test_image)
print(f'shape after going through conv layer: {test_image_through_conv.shape}')

#pass data through the max pool layer
test_image_through_max_pool_layer = max_pool_layer(test_image_through_conv)
print(f'shape after going through conv layer and max pool layer : {test_image_through_max_pool_layer.shape}')

test image original shape: torch.Size([3, 64, 64])
shape after going through conv layer: torch.Size([10, 62, 62])
shape after going through conv layer and max pool layer : torch.Size([10, 31, 31])


the max pool layer takes the maximum value and then compresses into a smaller vector space which contains the most important features

In [43]:
torch.manual_seed(42)

#create a random tensor with a similar number of dimensions of our images
random_tensor = torch.randn(size=(1,1,2,2))
print(f'\nrandom tensor:\n {random_tensor}')
print(f'random tensor shape: {random_tensor.shape}')


random tensor:
 tensor([[[[0.3367, 0.1288],
          [0.2345, 0.2303]]]])
random tensor shape: torch.Size([1, 1, 2, 2])


In [44]:
#create a max pool layer
max_pool_layer = nn.MaxPool2d(kernel_size=2)

#pass the random tensor through the max pool layer
max_pool_tensor= max_pool_layer(random_tensor)
print(f'\nMax Pool tensor\n: {max_pool_tensor}')
print(f'max pool tensor shape: {max_pool_tensor.shape}')
random_tensor


Max Pool tensor
: tensor([[[[0.3367]]]])
max pool tensor shape: torch.Size([1, 1, 1, 1])


tensor([[[[0.3367, 0.1288],
          [0.2345, 0.2303]]]])

#Layers in CNN architecture:

**1. Convolutional Layers (conv2d)**

Purpose:

* Convolutional layers are the core building blocks of CNNs. They are responsible for extracting features from the input data (e.g., edges, textures, shapes in images).
Operation:

* Convolutional layers apply a set of learnable filters (kernels) to the input image or feature map.

**2. ReLU Activation Layers**

Purpose:

* ReLU (Rectified Linear Unit) layers introduce non-linearity into the network, allowing it to learn complex patterns and functions.
Operation:

* ReLU applies an element-wise activation function to the input feature map.

* This means that all negative values in the feature map are set to zero, while positive values remain unchanged.

* Prevents vanishing gradient: Helps mitigate the vanishing gradient problem, making training more efficient.


**3. Pooling Layers (e.g., Max Pooling)**

Purpose:

* Pooling layers reduce the spatial dimensions (height and width) of the input feature maps, effectively downsampling them.

* This reduces the computational load and helps in achieving spatial invariance (robustness to small translations of the input).

#Training the CNN

In [45]:
from helper_functions import accuracy_fn

In [46]:
#setup a loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_2.parameters(),
                            lr=0.1)

In [47]:
#training and testing model2

torch.manual_seed(42)

#measure time
from timeit import default_timer as timer
train_time_start_model_2 = timer()

#train and test model
epochs = 3
for epoch in tqdm(range(epochs)):
  print(f'epoch : {epoch}\n')
  train_step(model=model_2,
             data_loader = train_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             accuracy_fn = accuracy_fn,
             device = device)
  test_step(model=model_2,
            data_loader = test_dataloader,
            loss_fn = loss_fn,
            accuracy_fn = accuracy_fn,
            device = device)

train_time_end_model_2 = timer()
total_train_time_model_2 = print_train_time(start=train_time_start_model_2,
                                            end=train_time_end_model_2,
                                            device=device)



  0%|          | 0/3 [00:00<?, ?it/s]

epoch : 0

TRAIN LOSS: 0.59518 | TRAIN ACC: 78.38
Test loss: 0.00157 | Test Acc:  0.26
Test loss: 0.00154 | Test Acc:  0.25
Test loss: 0.00117 | Test Acc:  0.28
Test loss: 0.00061 | Test Acc:  0.32
Test loss: 0.00131 | Test Acc:  0.28
Test loss: 0.00057 | Test Acc:  0.30
Test loss: 0.00069 | Test Acc:  0.31
Test loss: 0.00142 | Test Acc:  0.24
Test loss: 0.00068 | Test Acc:  0.32
Test loss: 0.00095 | Test Acc:  0.27
Test loss: 0.00112 | Test Acc:  0.27
Test loss: 0.00115 | Test Acc:  0.27
Test loss: 0.00139 | Test Acc:  0.27
Test loss: 0.00083 | Test Acc:  0.29
Test loss: 0.00165 | Test Acc:  0.26
Test loss: 0.00089 | Test Acc:  0.30
Test loss: 0.00078 | Test Acc:  0.30
Test loss: 0.00160 | Test Acc:  0.23
Test loss: 0.00111 | Test Acc:  0.28
Test loss: 0.00192 | Test Acc:  0.25
Test loss: 0.00112 | Test Acc:  0.29
Test loss: 0.00141 | Test Acc:  0.26
Test loss: 0.00150 | Test Acc:  0.28
Test loss: 0.00147 | Test Acc:  0.27
Test loss: 0.00123 | Test Acc:  0.28
Test loss: 0.00116 | Test

In [48]:
#Get model 2 results
model_2_results = eval_model(
    model=model_2,
    data_loader = test_dataloader,
    loss_fn = loss_fn,
    accuracy_fn = accuracy_fn,
    #device = device
)

model_2_results

  0%|          | 0/313 [00:00<?, ?it/s]

{'model_name:': 'FashionMNISTModelV2',
 'model_loss:': 0.32718968391418457,
 'model_acc:': 88.06908945686901}

#Comparing the model results and training results

In [49]:
import pandas as pd

compare_results = pd.DataFrame([model_0_results,
                               model_1_results,
                               model_2_results])

compare_results

,model_name:,model_loss:,model_acc:
0,FashionMNISTModel,0.476639,83.426518
1,FashionMNISTModelV1,0.685001,75.019968
2,FashionMNISTModelV2,0.327190,88.069089


In [50]:
#add training time to results comparision
compare_results['training time'] = [total_train_time_model0,
                                    total_train_time_model_1,
                                    total_train_time_model_2]
compare_results

,model_name:,model_loss:,model_acc:,training time
0,FashionMNISTModel,0.476639,83.426518,33.585537
1,FashionMNISTModelV1,0.685001,75.019968,35.184739
2,FashionMNISTModelV2,0.327190,88.069089,171.750402
